# Bit?cora de Cambios - EntremientoQuarto

**Nombre:** Francis Bravo  
**Fecha:** 14/02/2026  
**Proyecto:** `EntremientoQuarto`

## Objetivo de la bit?cora
Registrar de forma did?ctica todos los cambios aplicados para: configuraci?n de entorno, compatibilidad con Quartopy, continuidad de entrenamiento y robustez de visualizaci?n.

## ?ndice
1. [Formato de registro](#Formato-de-registro)
2. [Entrada 01 - Carga de `.env` y ruta de Quartopy](#Entrada-01---Carga-de-.env-y-ruta-de-Quartopy)
3. [Entrada 02 - Imports robustos (`utils` / `models`) en subcarpetas](#Entrada-02---Imports-robustos-utils--models-en-subcarpetas)
4. [Entrada 03 - Compatibilidad de `play_games` entre versiones de Quartopy](#Entrada-03---Compatibilidad-de-play_games-entre-versiones-de-Quartopy)
5. [Entrada 04 - No guardar partidas durante entrenamiento](#Entrada-04---No-guardar-partidas-durante-entrenamiento)
6. [Entrada 05 - Guardado de agentes fuera del repo](#Entrada-05---Guardado-de-agentes-fuera-del-repo)
7. [Entrada 06 - Correcci?n de gr?ficas (`Board.plot`)](#Entrada-06---Correcci?n-de-gr?ficas-Boardplot)
8. [Entrada 07 - Script para reanudar desde ?ltimo agente](#Entrada-07---Script-para-reanudar-desde-?ltimo-agente)
9. [Entrada 08 - Publicaci?n de `.env` en Git](#Entrada-08---Publicaci?n-de-.env-en-Git)
10. [Celdas de verificaci?n did?ctica](#Celdas-de-verificaci?n-did?ctica)

## Formato de registro
Cada entrada usa este formato:
- **Responsable:** Francis Bravo
- **Fecha:** 14/02/2026
- **Problema detectado**
- **Soluci?n aplicada**
- **Archivos y puntos exactos (hiperv?nculos por l?nea)**
- **Resultado esperado**

## Entrada 01 - Carga de `.env` y ruta de Quartopy
- **Responsable:** Francis Bravo
- **Fecha:** 14/02/2026
- **Problema detectado:** `ModuleNotFoundError: No module named 'quartopy'`.
- **Soluci?n aplicada:** se cre? un bootstrap centralizado para cargar `.env` y registrar `QUARTOPY_PATH` en `sys.path`.
- **Archivos y puntos exactos:**
  - [`utils/env_bootstrap.py#L17`](utils/env_bootstrap.py#L17)
  - [`trainRL.py#L4`](trainRL.py#L4)
  - [`trainRL.py#L7`](trainRL.py#L7)
  - [`bot/CNN_bot.py#L26`](bot/CNN_bot.py#L26)
  - [`QuartoRL/RL_functions.py#L20`](QuartoRL/RL_functions.py#L20)
- **Resultado esperado:** cualquier entrypoint puede resolver Quartopy usando variables del `.env`.

## Entrada 02 - Imports robustos (`utils` / `models`) en subcarpetas
- **Responsable:** Francis Bravo
- **Fecha:** 14/02/2026
- **Problema detectado:** ejecuci?n desde subcarpetas fallaba con `No module named utils/models`.
- **Soluci?n aplicada:** se a?adi? `PROJECT_ROOT` + `sys.path.insert(...)` antes de imports internos en m?dulos sensibles.
- **Archivos y puntos exactos:**
  - [`bot/human.py#L16`](bot/human.py#L16)
  - [`bot/random_bot.py#L16`](bot/random_bot.py#L16)
  - [`bot/CNN_F_bot.py#L19`](bot/CNN_F_bot.py#L19)
  - [`QuartoRL/contest.py#L14`](QuartoRL/contest.py#L14)
  - [`QuartoRL/observers.py#L19`](QuartoRL/observers.py#L19)
  - [`models/CNN1.py#L18`](models/CNN1.py#L18)
  - [`models/CNN_uncoupled.py#L18`](models/CNN_uncoupled.py#L18)
  - [`models/CNN_fdec.py#L12`](models/CNN_fdec.py#L12)
- **Resultado esperado:** imports estables sin depender del directorio de ejecuci?n.

## Entrada 03 - Compatibilidad de `play_games` entre versiones de Quartopy
- **Responsable:** Francis Bravo
- **Fecha:** 14/02/2026
- **Problema detectado:** diferencias de firma en `play_games` (`save_match`, `mode_2x2`, tipos de retorno).
- **Soluci?n aplicada:** wrapper de compatibilidad con introspecci?n de firma y normalizaci?n de resultados.
- **Archivos y puntos exactos:**
  - [`utils/play_games_compat.py#L6`](utils/play_games_compat.py#L6)
  - [`utils/play_games_compat.py#L48`](utils/play_games_compat.py#L48)
  - [`utils/play_games_compat.py#L68`](utils/play_games_compat.py#L68)
  - [`QuartoRL/RL_functions.py#L21`](QuartoRL/RL_functions.py#L21)
  - [`QuartoRL/contest.py#L19`](QuartoRL/contest.py#L19)
  - [`run_swiss_tournament.py#L43`](run_swiss_tournament.py#L43)
- **Resultado esperado:** entrenamiento y evaluaci?n funcionando aunque cambie la API de Quartopy.

## Entrada 04 - No guardar partidas durante entrenamiento
- **Responsable:** Francis Bravo
- **Fecha:** 14/02/2026
- **Problema detectado:** se estaban exportando CSV de partidas en procesos donde no se deseaba persistencia.
- **Soluci?n aplicada:** `play_games_compat` desactiva temporalmente export cuando `save_match=False`.
- **Archivos y puntos exactos:**
  - [`utils/play_games_compat.py#L48`](utils/play_games_compat.py#L48)
  - [`utils/play_games_compat.py#L133`](utils/play_games_compat.py#L133)
  - [`QuartoRL/RL_functions.py#L258`](QuartoRL/RL_functions.py#L258)
  - [`QuartoRL/contest.py#L131`](QuartoRL/contest.py#L131)
  - [`play_between_bots.py#L108`](play_between_bots.py#L108)
- **Resultado esperado:** no se generan archivos de partida durante self-play y contest, pero s? se conserva entrenamiento.

## Entrada 05 - Guardado de agentes fuera del repo
- **Responsable:** Francis Bravo
- **Fecha:** 14/02/2026
- **Problema detectado:** necesidad de guardar checkpoints en ruta externa (`...\Mech Interp\CHECKPOINTS`).
- **Soluci?n aplicada:** `CHECKPOINTS_ROOT` configurable v?a `.env` y usado por `trainRL`.
- **Archivos y puntos exactos:**
  - [`trainRL.py#L45`](trainRL.py#L45)
  - [`trainRL.py#L48`](trainRL.py#L48)
  - [`.env#L8`](.env#L8)
- **Resultado esperado:** agentes guardados fuera del repositorio de c?digo.

## Entrada 06 - Correcci?n de gr?ficas (`Board.plot`)
- **Responsable:** Francis Bravo
- **Fecha:** 14/02/2026
- **Problema detectado:** crash en ?poca 50 por `AttributeError: 'Board' object has no attribute 'plot'`.
- **Soluci?n aplicada:** fallback visual con Matplotlib a partir de `to_matrix()/encode()`.
- **Archivos y puntos exactos:**
  - [`QuartoRL/observers.py#L35`](QuartoRL/observers.py#L35)
  - [`QuartoRL/observers.py#L62`](QuartoRL/observers.py#L62)
  - [`QuartoRL/observers.py#L165`](QuartoRL/observers.py#L165)
- **Resultado esperado:** las gr?ficas ya no se rompen por diferencias de versi?n de `Board`.

## Entrada 07 - Script para reanudar desde ?ltimo agente
- **Responsable:** Francis Bravo
- **Fecha:** 14/02/2026
- **Problema detectado:** `trainRL.py` inicia en random y no contin?a autom?ticamente.
- **Soluci?n aplicada:** se cre? `trainRL_resume_latest.py` con auto-detecci?n del checkpoint m?s reciente y continuidad de numeraci?n de ?pocas.
- **Archivos y puntos exactos:**
  - [`trainRL_resume_latest.py#L44`](trainRL_resume_latest.py#L44)
  - [`trainRL_resume_latest.py#L129`](trainRL_resume_latest.py#L129)
  - [`trainRL_resume_latest.py#L143`](trainRL_resume_latest.py#L143)
  - [`trainRL_resume_latest.py#L219`](trainRL_resume_latest.py#L219)
  - [`trainRL_resume_latest.py#L284`](trainRL_resume_latest.py#L284)
  - [`trainRL_resume_latest.py#L420`](trainRL_resume_latest.py#L420)
- **Resultado esperado:** reanudar entrenamiento desde el ?ltimo agente sin sobrescribir la l?gica de entrenamiento principal.

## Entrada 08 - Publicaci?n de `.env` en Git
- **Responsable:** Francis Bravo
- **Fecha:** 14/02/2026
- **Problema detectado:** `.env` estaba ignorado por Git.
- **Soluci?n aplicada:** excepci?n expl?cita en `.gitignore`.
- **Archivos y puntos exactos:**
  - [`.gitignore#L127`](.gitignore#L127)
  - [`.env#L2`](.env#L2)
  - [`.env#L5`](.env#L5)
- **Resultado esperado:** `.env` versionable en GitHub seg?n decisi?n del proyecto.

## Celdas de verificaci?n did?ctica
Las siguientes celdas son ejemplos para validar r?pidamente que los cambios se comportan como se espera.

In [ ]:
# Verificaci?n 1: carga de entorno y Quartopy
from pathlib import Path
from utils.env_bootstrap import bootstrap_quartopy_path

project_root = Path('.').resolve()
resolved = bootstrap_quartopy_path(project_root)
print('Ruta Quartopy resuelta:', resolved)

In [ ]:
# Verificaci?n 2: compatibilidad de play_games
from bot.random_bot import Quarto_bot as RandomBot
from utils.play_games_compat import play_games_compat

payload, wr = play_games_compat(
    matches=2,
    player1=RandomBot(),
    player2=RandomBot(),
    verbose=False,
    save_match=False,
)
print('Win-rate normalizado:', wr)
print('Tipo payload:', type(payload).__name__)

In [ ]:
# Verificaci?n 3: detecci?n de ?ltimo checkpoint (l?gica resumida)
from pathlib import Path
import re

ckpt_dir = Path(r"C:/Users/bravo/Documents/Metodos Numericos Pycharm/Mech Interp/CHECKPOINTS/05_LOSS")

def extract_epoch(name: str) -> int:
    m = re.findall(r"(?:_E_|epoch_)(\d+)", name, flags=re.IGNORECASE)
    return int(m[-1]) if m else -1

pts = list(ckpt_dir.glob('*.pt'))
if pts:
    latest = max(pts, key=lambda p: (extract_epoch(p.stem), p.stat().st_mtime))
    print('?ltimo checkpoint:', latest)
else:
    print('No hay checkpoints en la carpeta')

## Nota final
Esta bit?cora est? orientada a mantenimiento y docencia: cada ajuste relevante qued? trazado con enlace por l?nea, problema original y efecto esperado.